# import 

In [25]:
import cv2

In [26]:
import random

In [ ]:
from fastai.vision import *

In [13]:
import os

In [151]:
# export
from fastprogress.fastprogress import progress_bar

# functions

In [145]:
# 关注的水体的灰度值。
#0 未标注。 1 空地。 2 建筑 。3 水体。4 道路
TYP_WATER = 3

In [146]:
#水体占比在这个值一下的认为是空的
BLANK_TH = 0.04

In [159]:
def gen_dataset(imgfns, maskfns, output_dir = 'label/dataset_20200708', imgdir = 'image'
                , maskdir = 'mask', img_sz = (512, 512), ds_sz = 1000, blank_pct = 0.2
                , suffix = '.png', dbg = []):
    '''
    生成数据集
    参数：
        imgfns：大图片文件名列表
        maskfns：对应的mask文件名列表。与上一个参数里面的文件一一对应
        output_dir：输出数据集目录
        imgdir：输出数据集中存放图片的子目录。
        maskdir：输出数据集中存放mask图片的子目录
        img_sz：提取的小图的大小,内容为(width, height)
        ds_sz：生成图片数量
        blank_pct：无水体图片数量比例
        suffix：生成图片文件扩展名
        dbg：调试用
    '''
    assert len(imgfns) == len(maskfns)
    #检查创建目录
    for sdir in ['', imgdir, maskdir]:
        if not os.path.exists(os.path.join(output_dir, sdir)):
            os.makedirs(os.path.join(output_dir, sdir))
    total_cnt = 0#产出计数
    imgs = []
    for imgfn, maskfn in zip(imgfns, maskfns):
        img = cv2.imread(imgfn)
        mask = (cv2.imread(maskfn, cv2.IMREAD_GRAYSCALE) == TYP_WATER).astype(int) * 255
        #print(imgfn, maskfn, img.shape, mask.shape)
        assert img.shape[ : 2] == mask.shape[ : 2]
        imgs += [(img, mask)]
    
    #当前大图需要产生的空白数据图片的数量
    DS_SZ_BLANK = int(ds_sz * blank_pct)        
    #需要产生的有效图片的数量
    DS_SZ_VALID = ds_sz - DS_SZ_BLANK

    validcnt = 0 #当前图片产生的数量
    blankcnt = 0#当前产出的空白数据集的数量
    #print('dbg1', DS_SZ_1IMG, DS_SZ_1IMG_BLANK, DS_SZ_1IMG_VALID)
    
    pbar = progress_bar(range(ds_sz))
    pbar.comment = '生成中'
    for p in pbar:
        while validcnt < DS_SZ_VALID or blankcnt < DS_SZ_BLANK:
            idx = random.randint(0, 2)
            #print(idx, imgfns[idx])
            img, mask = imgs[idx]
            dsx = random.randint(0, img.shape[0] - img_sz[1])
            dsy = random.randint(0, img.shape[1] - img_sz[0])
            dsimg = img[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]

            dsmask = mask[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]        
            area = cv2.countNonZero(dsmask)
            #根据比例判断一下是否有效的
            if area / (img_sz[0] * img_sz[1]) > BLANK_TH:
                #print(validcnt, total_cnt, ds_sz % len(imgfns))
                #超过了数量的放弃.并且保证到达整数
                if validcnt >= DS_SZ_VALID:
                    continue
                validcnt += 1
            else:
                if blankcnt >= DS_SZ_BLANK:
                    continue
                blankcnt += 1

            cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (total_cnt, suffix.split('.')[-1])), dsimg)
            cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (total_cnt, suffix.split('.')[-1])), dsmask)

            total_cnt += 1
            break


# test

In [160]:
imgfns = ['data/src/image/1.png', 'data/src/image/2.png', 'data/src/image/3.png']

In [161]:
maskimgfns = ['data/label/train1_labels_8bits.png', 'data/label/train2_labels_8bits.png'
             , 'data/label/train3_labels_8bits.png']

In [163]:
gen_dataset(imgfns, maskimgfns, ds_sz = 2000, suffix='jpg') 

# export